In [72]:
import random
import numpy
game_length = 14

current_pos = -1

step_count = 0

step_counts=[]

for i in range(100000):

    while current_pos <= game_length:
        step_count += 1
        # print("move to:" + str(current_pos))
        current_pos+= random.choice([0,0,1,1])+random.choice([0,0,1,1])+random.choice([0,0,1,1])

    # print("steps:" + str(step_count))
    current_pos =-1
    step_counts.append(step_count)
    step_count = 0

step_counts


[9,
 12,
 10,
 13,
 16,
 12,
 9,
 9,
 11,
 13,
 9,
 15,
 12,
 7,
 11,
 8,
 11,
 9,
 11,
 18,
 9,
 9,
 9,
 14,
 9,
 13,
 8,
 12,
 12,
 17,
 10,
 10,
 9,
 11,
 9,
 13,
 10,
 11,
 10,
 8,
 10,
 10,
 14,
 10,
 10,
 9,
 12,
 11,
 13,
 10,
 8,
 12,
 12,
 11,
 10,
 14,
 13,
 11,
 12,
 9,
 9,
 12,
 9,
 10,
 7,
 11,
 13,
 15,
 9,
 11,
 13,
 12,
 9,
 14,
 11,
 11,
 8,
 9,
 12,
 11,
 10,
 11,
 12,
 11,
 11,
 14,
 13,
 11,
 9,
 11,
 11,
 12,
 10,
 11,
 9,
 13,
 10,
 9,
 11,
 9,
 10,
 8,
 14,
 15,
 12,
 11,
 11,
 11,
 15,
 10,
 11,
 11,
 9,
 9,
 11,
 12,
 11,
 13,
 9,
 14,
 10,
 7,
 12,
 12,
 10,
 12,
 10,
 10,
 10,
 10,
 12,
 11,
 10,
 12,
 8,
 12,
 13,
 12,
 12,
 12,
 15,
 10,
 11,
 10,
 11,
 10,
 12,
 11,
 11,
 14,
 10,
 12,
 11,
 12,
 13,
 13,
 12,
 11,
 10,
 11,
 12,
 11,
 15,
 15,
 14,
 9,
 11,
 8,
 11,
 9,
 11,
 10,
 12,
 11,
 10,
 10,
 11,
 10,
 15,
 13,
 14,
 16,
 11,
 9,
 11,
 11,
 13,
 14,
 12,
 10,
 12,
 18,
 10,
 9,
 10,
 11,
 11,
 12,
 14,
 15,
 11,
 10,
 8,
 15,
 13,
 10,
 10,
 9,
 1

In [73]:
step_counts.sort()
print("max:" + str(step_counts[-1]))
print("min:" + str(step_counts[0]))
numpy.mean(step_counts)

max:24
min:6


10.99288

In [ ]:
game_length = 14

current_pos = -1

step_count = 0



while current_pos <= game_length:
       step_count += 1
       for i in range(current_pos):
              print(" ", end='')
       print("┠", end='')
       move_dist = random.choice([0,0,1,1])+random.choice([0,0,1,1])+random.choice([0,0,1,1])
       for i in range(move_dist-1):
              print("─", end='')
       print("┨ "+ str(move_dist) )

       current_pos+= move_dist

       print("┠╂", end='')
       for i in range(game_length-1):
              print("┬", end='')
       print("╂┨ "+str(current_pos))
       for i in range(current_pos):
              print(" ", end='')
       print("┃")

 
